In [1]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
CUDA_VISIBLE_DEVICES=1
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

import numpy as np
import pandas as pd
import pickle
from tensorflow import keras

from tensorflow.keras.metrics import RootMeanSquaredError as rmse
from tensorflow.keras.models import load_model
from keras import regularizers
from keras import backend
from keras.layers import Dropout, Input, BatchNormalization, concatenate, Activation, MaxPooling2D, AveragePooling2D, Flatten, Masking
from keras.layers import Bidirectional, GRU, Conv2D, TimeDistributed, LSTM, Dense
from keras.models import Model, Sequential

def RMSE(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true)))

def BIAS(y_true, y_pred):
	return backend.mean((y_pred - y_true), axis=-1)


Num GPUs Available:  0
Num GPUs Available:  []


In [2]:
data = np.load('mlp_data.npz',allow_pickle=True)
mlp_X,mlp_Y = data['arr_0'], data['arr_1']

def build_model_mlp(trainX):
    model=Sequential()
    model.add(Dense(25,activation='relu',input_shape=(trainX.shape[1],)))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=[RMSE,BIAS])
    return model

model=build_model_mlp(mlp_X)
history=model.fit(mlp_X,mlp_Y,epochs=20,verbose=1)

Epoch 1/20


2023-03-13 00:28:45.854057: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


313/313 [==============================] - 1s 632us/step - loss: 268655.5000 - RMSE: 515.7129 - BIAS: -425.1102
Epoch 2/20
313/313 [==============================] - 0s 590us/step - loss: 176332.6719 - RMSE: 414.0849 - BIAS: -312.4889
Epoch 3/20
313/313 [==============================] - 0s 627us/step - loss: 54175.3984 - RMSE: 227.1171 - BIAS: -66.9613
Epoch 4/20
313/313 [==============================] - 0s 612us/step - loss: 30429.7949 - RMSE: 171.5508 - BIAS: 3.5552
Epoch 5/20
313/313 [==============================] - 0s 611us/step - loss: 22413.1660 - RMSE: 145.6960 - BIAS: 2.4099
Epoch 6/20
313/313 [==============================] - 0s 609us/step - loss: 17221.2246 - RMSE: 126.8211 - BIAS: 0.0605
Epoch 7/20
313/313 [==============================] - 0s 602us/step - loss: 14926.2285 - RMSE: 117.5199 - BIAS: -2.3996
Epoch 8/20
313/313 [==============================] - 0s 617us/step - loss: 13653.4785 - RMSE: 112.5108 - BIAS: -2.1635
Epoch 9/20
313/313 [===========================

In [3]:
data = np.load('bigru_data.npz',allow_pickle=True)
bigru_X,bigru_Y = data['arr_0'], data['arr_1']

def build_model_bigru(trainX):
    model = Sequential()
    model.add(Bidirectional(GRU(32,  input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)))
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=5, activation='relu'))
    model.add(Dense(units=1))
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='mse',metrics=[RMSE,BIAS])
    return model

model=build_model_bigru(bigru_X)
history=model.fit(bigru_X,bigru_Y, epochs=30)

Epoch 1/30


/home/lir/.conda/envs/lenv2/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


10/10 [==============================] - 2s 24ms/step - loss: 201796.1094 - RMSE: 450.5330 - BIAS: -326.9475
Epoch 2/30
10/10 [==============================] - 0s 23ms/step - loss: 200954.5000 - RMSE: 444.2608 - BIAS: -325.6695
Epoch 3/30
10/10 [==============================] - 0s 23ms/step - loss: 199522.3125 - RMSE: 448.9027 - BIAS: -323.8145
Epoch 4/30
10/10 [==============================] - 0s 23ms/step - loss: 197283.7500 - RMSE: 441.4278 - BIAS: -320.9561
Epoch 5/30
10/10 [==============================] - 0s 23ms/step - loss: 193914.0469 - RMSE: 443.6653 - BIAS: -316.1460
Epoch 6/30
10/10 [==============================] - 0s 23ms/step - loss: 189405.9844 - RMSE: 435.5395 - BIAS: -308.9622
Epoch 7/30
10/10 [==============================] - 0s 22ms/step - loss: 184060.4531 - RMSE: 432.3969 - BIAS: -300.2178
Epoch 8/30
10/10 [==============================] - 0s 21ms/step - loss: 178463.4062 - RMSE: 422.3336 - BIAS: -290.8792
Epoch 9/30
10/10 [==============================] -

In [4]:
data = np.load('densenet_data.npz',allow_pickle=True)
densenet_X,densenet_Y = data['arr_0'], data['arr_1']

def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters


def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters


def dense_net(in_shape,out_shape,filters, growth_rate, dense_block_size, layers_in_block):
    input_img = Input(shape=in_shape)
    x = Conv2D(24, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = Flatten()(dense_x)

    output = Dense(out_shape, activation='relu')(dense_x)

    return Model(input_img, output)

dense_block_size = 3
layers_in_block = 3
growth_rate = 4
model_merge = dense_net((densenet_X.shape[1],densenet_X.shape[2],densenet_X.shape[3]),1,growth_rate * 2, growth_rate, dense_block_size, layers_in_block)

optimizer = keras.optimizers.RMSprop(lr=0.001)
model_merge.compile(optimizer = optimizer, loss='mse',metrics=[RMSE,BIAS])

batch_size = 32
epochs = 15
history=model_merge.fit(densenet_X,densenet_Y, epochs=epochs, batch_size=batch_size)


/home/lir/.conda/envs/lenv2/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/15
313/313 [==============================] - 7s 17ms/step - loss: 234186.3438 - RMSE: 479.7634 - BIAS: -408.2061
Epoch 2/15
313/313 [==============================] - 5s 17ms/step - loss: 108363.2422 - RMSE: 320.9772 - BIAS: -274.7201
Epoch 3/15
313/313 [==============================] - 5s 17ms/step - loss: 25192.1641 - RMSE: 151.1797 - BIAS: -83.9304
Epoch 4/15
313/313 [==============================] - 5s 17ms/step - loss: 14184.8193 - RMSE: 115.5281 - BIAS: -6.1809
Epoch 5/15
313/313 [==============================] - 5s 17ms/step - loss: 12859.2861 - RMSE: 110.1476 - BIAS: -1.9900
Epoch 6/15
313/313 [==============================] - 5s 17ms/step - loss: 12815.0381 - RMSE: 109.7067 - BIAS: -1.3856
Epoch 7/15
313/313 [==============================] - 5s 17ms/step - loss: 12559.3398 - RMSE: 108.4762 - BIAS: -1.4636
Epoch 8/15
313/313 [==============================] - 5s 17ms/step - loss: 11815.2305 - RMSE: 105.1498 - BIAS: -1.2818
Epoch 9/15
313/313 [=====================

In [5]:
data = np.load('cnngru_data.npz',allow_pickle=True)
cnngru_X,cnngru_Y = data['arr_0'], data['arr_1']

def build_model_cnngru(trainX):
    shape = (trainX.shape[1], trainX.shape[2], trainX.shape[3], trainX.shape[4])
    nLstmNeu=32

    model = Sequential()
    # --- CNN
    model.add(TimeDistributed(Conv2D(filters=16,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.01),
                                     activation='relu',input_shape=shape)))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(filters=32,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.01),
                                     activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(l=0.1),
                                      activation='relu')))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Flatten()))
    #model.add(TimeDistributed(Dropout(0.5)))

    # --- LSTM
    model.add(Bidirectional(GRU(nLstmNeu, return_sequences=True, activation='tanh')))
    model.add(TimeDistributed(Dense(8, activation='relu',
                                    kernel_initializer="glorot_uniform",kernel_regularizer=regularizers.l2(l=0.01))))
    model.add(TimeDistributed(Dense(1, activation='relu',
                                    kernel_initializer="glorot_uniform",kernel_regularizer=regularizers.l2(l=0.01))))
    # --- Optimizer 
    adam = keras.optimizers.Adam(lr=0.0005)
    model.compile(optimizer=adam, loss='mse',metrics=[RMSE,BIAS])
    
    return model

model=build_model_cnngru(cnngru_X)
history=model.fit(cnngru_X,cnngru_Y, batch_size=8,epochs=30)#10


Epoch 1/30
38/38 [==============================] - 5s 82ms/step - loss: 197526.3906 - RMSE: 439.6683 - BIAS: -321.5135
Epoch 2/30
38/38 [==============================] - 3s 81ms/step - loss: 194053.7812 - RMSE: 437.2147 - BIAS: -316.6888
Epoch 3/30
38/38 [==============================] - 3s 81ms/step - loss: 191836.5625 - RMSE: 436.2387 - BIAS: -314.4688
Epoch 4/30
38/38 [==============================] - 3s 84ms/step - loss: 189521.7656 - RMSE: 431.1381 - BIAS: -311.8939
Epoch 5/30
38/38 [==============================] - 3s 82ms/step - loss: 187043.6562 - RMSE: 429.9130 - BIAS: -308.9781
Epoch 6/30
38/38 [==============================] - 3s 83ms/step - loss: 184420.8750 - RMSE: 427.2227 - BIAS: -305.7776
Epoch 7/30
38/38 [==============================] - 3s 84ms/step - loss: 181634.2812 - RMSE: 420.5208 - BIAS: -302.3031
Epoch 8/30
38/38 [==============================] - 3s 80ms/step - loss: 178706.3125 - RMSE: 418.8466 - BIAS: -298.5723
Epoch 9/30
38/38 [======================

In [6]:
model=load_model('cnngru_model.h5',custom_objects={'RMSE':RMSE,'BIAS':BIAS})